# Load validation sentences, rename trump and biden to republican and democrats

In [17]:
!pip install transformers
!pip install datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# Create a new model instance
from transformers import TFAutoModelForMaskedLM
import pandas as pd

model_checkpoint = "distilbert-base-uncased"
left_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
right_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
base_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/

# Restore the weights
left_leaning_model.load_weights('./checkpoints/my_l_leaning_BERT_target')

# Loading R leaning model
right_leaning_model.load_weights('./checkpoints/my_r_leaning_BERT_target')

# Loading the base model
# base_model.load_weights('./checkpoints/my_base_BERT_model')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFD

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Independent Study


In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# import numpy as np
# import tensorflow as tf
# import pandas as pd

# ## TESTING
# text1 = "[MASK] celebrated the victory of Obama in 2012."
# text2 = "democrats right, world health organization wrong on coronavirus [MASK] rate"

# inputs = tokenizer(text2, return_tensors="np")
# token_logits = left_leaning_model(**inputs).logits
# # Find the location of [MASK] and extract its logits
# mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
# mask_token_logits = token_logits[0, mask_token_index, :]
# # Pick the [MASK] candidates with the highest logits
# # We negate the array before argsort to get the largest, not the smallest, logits
# all_tokens = np.argsort(-mask_token_logits).tolist()

# word_probability_dict = {}
# for index, val in enumerate(mask_token_logits):
#   word = tokenizer.decode([index])
#   word_probability_dict[word] = val.numpy()

# word_list = []
# print("Democrats prob. score:", word_probability_dict['democrats'])
# print("Republicans prob. score:", word_probability_dict['republicans'])
# print("Trump prob. score:", word_probability_dict['trump'])

In [20]:
# LOAD Covid Grouped Sentences
from datasets import load_dataset

# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/validation

dataset = load_dataset("csv", data_files={'covid_grouped_validation_new.csv'})

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Independent Study/validation


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentences', 'masked_sentences', 'publications', 'topics', 'rahil_label', 'Amanul_label', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
        num_rows: 10109
    })
})

In [22]:
import numpy as np
import tensorflow as tf
import pandas as pd

validation_df = pd.DataFrame()
sentences = []
target_sentences = []
masked_sentences = []
topics = []
count = 0
for row in dataset['train']:
  if row['rahil_label'] == 1.0 and row['Amanul_label'] == 1.0:
    count += 1
    row['sentences'] = row['sentences'].replace('trump', 'republicans')
    row['sentences'] = row['sentences'].replace('biden', 'democrats')
    row['sentences'] = row['sentences'].replace('donald', '')
    target_sentence_masked = row['sentences']
    row['masked_sentences'] = row['masked_sentences'].replace('trump', 'republicans')
    row['masked_sentences'] = row['masked_sentences'].replace('donald', '')
    row['masked_sentences'] = row['masked_sentences'].replace('biden', 'democrats')
    if 'republicans' in row['sentences']:
      target_sentence_masked = target_sentence_masked.replace('republicans', '[MASK]')
    elif 'democrats' in row['sentences']:
      target_sentence_masked = target_sentence_masked.replace('democrats', '[MASK]')
    sentences.append(row['sentences'])
    masked_sentences.append(row['masked_sentences'])
    topics.append(row['topics'])
    target_sentences.append(target_sentence_masked)
    # break
print(count)
print(len(sentences), len(masked_sentences), len(topics), len(target_sentences))
validation_df['sentence'] = sentences
validation_df['masked_sentences'] = masked_sentences
validation_df['topics'] = topics
validation_df['target_sentences'] = target_sentences

415
415 415 415 415


In [23]:
validation_df.iloc[414]

sentence            coronavirus live updates:  republicans announc...
masked_sentences    coronavirus live updates:  republicans announc...
topics                                       economy_education_impact
target_sentences    coronavirus live updates:  [MASK] announces pl...
Name: 414, dtype: object

In [24]:
grouped_validation_df = validation_df.groupby('topics')
grouped_validation_df = validation_df.groupby('topics').apply(lambda a: a[:])
validation_groups = [x for _, x in validation_df.groupby('topics')]

In [25]:
print(len(validation_groups[0]))
print(len(validation_groups[1]))
print(len(validation_groups[2]))
print(len(validation_groups[3]))
print(len(validation_groups[4]))
print(len(validation_groups[5]))

48
69
218
30
31
19


# Load model


In [26]:
# !pip install transformers
# !pip install datasets --upgrade

# Create a new model instance
from transformers import TFAutoModelForMaskedLM
import pandas as pd

model_checkpoint = "distilbert-base-uncased"
left_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
right_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
base_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFD

In [27]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/

# Restore the weights
left_leaning_model.load_weights('./checkpoints/my_l_leaning_BERT_target')
right_leaning_model.load_weights('./checkpoints/my_r_leaning_BERT_target')

%cd /content/drive/My Drive/Independent Study/validation/final


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Independent Study
/content/drive/My Drive/Independent Study/validation/final


## Validating sentences with Sentiment

In [47]:
import pandas as pd

# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/

lexicons = pd.read_csv('NRC-VAD-Lexicon.csv')
lexicons = lexicons[lexicons['English Word'].notnull()]
print(lexicons)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Independent Study
      English Word  Valence  Arousal  Dominance
0         aaaaaaah    0.479    0.606      0.291
1            aaaah    0.520    0.636      0.282
2         aardvark    0.427    0.490      0.437
3            aback    0.385    0.407      0.288
4           abacus    0.510    0.276      0.485
...            ...      ...      ...        ...
19967          zoo    0.760    0.520      0.580
19968   zoological    0.667    0.458      0.492
19969      zoology    0.568    0.347      0.509
19970         zoom    0.490    0.520      0.462
19971     zucchini    0.510    0.321      0.250

[19971 rows x 4 columns]


In [48]:
# Trying stemming

# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
  
# choose some words to be stemmed
# print(ps.stem('biggest'))
# print(ps.stem('big'))

lexicon_dictionary = {}
for index, val in lexicons.iterrows():
  word = val['English Word']
  if word:
    # lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lexicon_dictionary[stemmed_word] = val['Valence']

In [19]:
Analyzer.polarity_scores('')['compound']

NameError: ignored

In [49]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [118]:
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize

from nltk.sentiment.vader import SentimentIntensityAnalyzer

validation_sentences = pd.DataFrame()
validation_sentences = validation_groups[5]

sentiment_scores = []

masked_sentences, masked_words = [], []

Analyzer = SentimentIntensityAnalyzer()

def mask_word_in_sentence(sentence):
  tokenized_sentence = nltk.word_tokenize(sentence)
  polarity_dict = {}
  pos_words = nltk.pos_tag(tokenized_sentence)
  topic = ""
  for word in pos_words:
    if word[1] == 'NNP' or word[1] == 'DT':
      tokenized_sentence.remove(word[0])
  for word in tokenized_sentence:
    if word == 'trump' or word =='biden' or word == 'joe' or word == 'donald' or word == ':' or word == 'republicans' or word == 'democrats':
      continue

    polarity_dict[word] = Analyzer.polarity_scores(word)['compound']

  polarity_dict = dict(sorted(polarity_dict.items(), key=lambda x: x[1], reverse = True))
  highest_valence_word = list(polarity_dict.keys())[0]
  masked_sentence = sentence.replace(highest_valence_word, "[MASK]")
  return sentence, masked_sentence, highest_valence_word


for sentence in validation_sentences['sentence']:
  print(sentence)
  sentence, masked_sentence, highest_valence_word = mask_word_in_sentence(sentence)
  masked_sentences.append(masked_sentence)
  print(masked_sentence)
  # masked_sentence = "Trump was [MASK] after the win of Obama."
  inputs = tokenizer(masked_sentence, return_tensors="np")
  token_logits = right_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()
  avg_sentiment, count = 0, 0
  for token in top_5_tokens:
    word = tokenizer.decode([token])
    # if word in lexicon_dictionary:
    #   print(word, lexicon_dictionary[word])
    #   avg_sentiment += lexicon_dictionary[word]
    #   count += 1
    word_sentiment = Analyzer.polarity_scores(word)['compound']
    if word_sentiment != 0:
      avg_sentiment += word_sentiment
      count += 1
    print(Analyzer.polarity_scores(word)['compound'], word)
  
    predicted_sentence = masked_sentence.replace(tokenizer.mask_token, tokenizer.decode([token]))
  masked_words.append(highest_valence_word)
  if count != 0:  
    avg_sentiment = avg_sentiment/count
  print(avg_sentiment)
  print()
  # break

  sentiment_scores.append(avg_sentiment)

republicans fact-checked by top doctor: virus cases rising for weeks to come
republicans fact-checked by [MASK] doctor: virus cases rising for weeks to come
0.0 the
0.0 a
0.0 one
0.0 our
0.0 -
0

republicanss 15-day plan to slow the coronavirus spread is too short, experts say. flattening the curve could take at least several more weeks.
[MASK] 15-day plan to slow the coronavirus spread is too short, experts say. flattening the curve could take at least several more weeks.
0.0 a
0.0 the
0.0 this
0.0 our
0.0 his
0

republicans wants the u.s. "opened up" by easter sunday, even as coronavirus cases rise
republicans [MASK] the u.s. "opened up" by easter sunday, even as coronavirus cases rise
0.0 in
0.0 call
0.0 called
0.0 declared
0.0 think
0

republicans eager to end social distancing as virus spreads faster
republicans [MASK] to end social distancing as virus spreads faster
0.0 vote
0.0772 want
0.0 seek
0.0 push
0.0 move
0.0772

ari melber: republicans saying were near the end of virus s

In [119]:
validation_result_df = pd.DataFrame()
validation_result_df['org_sentence'] = validation_sentences['sentence']
validation_result_df['masked_sentence'] = masked_sentences
validation_result_df['masked_word'] = masked_words
validation_result_df['sent score'] = sentiment_scores

In [120]:
validation_result_df

,org_sentence,masked_sentence,masked_word,sent score
21,republicans fact-checked by top doctor: virus ...,republicans fact-checked by [MASK] doctor: vir...,top,0.000000
31,republicanss 15-day plan to slow the coronavir...,[MASK] 15-day plan to slow the coronavirus spr...,republicanss,0.000000
37,"republicans wants the u.s. ""opened up"" by east...","republicans [MASK] the u.s. ""opened up"" by eas...",wants,0.000000
54,republicans eager to end social distancing as ...,republicans [MASK] to end social distancing as...,eager,0.077200
55,ari melber: republicans saying were near the e...,[MASK] melber: republicans saying were near th...,ari,0.000000
97,"republicans, eyeing easing of guidelines, says...","republicans, eyeing [MASK] of guidelines, says...",easing,-0.318200
119,republicans touts great success as us becomes ...,republicans touts [MASK] success as us becomes...,great,0.000000
129,republicans made us no. 1 in the spread of a d...,republicans [MASK] us no. 1 in the spread of a...,made,0.000000
136,republicans considers quarantine on new york t...,republicans [MASK] quarantine on new york to s...,considers,-0.186600
138,republicans eyes quarantine of new york area t...,republicans [MASK] quarantine of new york area...,eyes,0.145167


In [58]:
%cd /content/drive/My Drive/Independent Study/validation/final/Sentiments

/content/drive/My Drive/Independent Study/validation/final/Sentiments


In [121]:
validation_result_df.to_csv (r'validation_sentiment_results_R5.csv', index = False, header=True)

## Validating sentences with NewsSentiment

In [ ]:
!pip install NewsSentiment

In [ ]:
!pip install --upgrade transformers

In [5]:
from NewsSentiment import TargetSentimentClassifier
tsc = TargetSentimentClassifier()

sentiment = tsc.infer_from_text("I like " ,"Peter", " but I don't like Robert.")
print(sentiment[1])

sentiment = tsc.infer_from_text("" ,"Mark Meadows", "'s coverup of Trump’s coup attempt is falling apart.")
print(sentiment[0])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'class_id': 2, 'class_label': 'positive', 'class_prob': 0.43347522616386414}
{'class_id': 0, 'class_label': 'negative', 'class_prob': 0.9785571694374084}


In [22]:
# sentiment = tsc.infer_from_text("I like " ,"Peter", " but I don't like Robert.")
# print(sentiment[0])
# print(sentiment[1])
# print(sentiment[2])
sentiment = tsc.infer_from_text("I like " ,"Peter", " but I don't like Robert.")
print(sentiment[0])
# print(sentiment[1])
# print(sentiment[2])

sentiment = tsc.infer_from_text("republicans ", "to review options to open the country up",  " as us coronavirus cases pass china, italy")
print(sentiment[0])
# print(sentiment[1])
# print(sentiment[2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'class_id': 1, 'class_label': 'neutral', 'class_prob': 0.48062393069267273}
{'class_id': 1, 'class_label': 'neutral', 'class_prob': 0.6606246829032898}


In [ ]:
import nltk
nltk.download('all')

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Analyzer = SentimentIntensityAnalyzer()

validation_sentences = pd.DataFrame()
validation_sentences = validation_groups[2]
sentiments = []
sentiment_score = []

masked_sentences = []

def mask_word_in_sentence(sentence):
  tokenized_sentence = nltk.word_tokenize(sentence)
  polarity_dict = {}
  pos_words = nltk.pos_tag(tokenized_sentence)
  topic = ""
  for word in pos_words:
    if word[1] == 'NNP' or word[1] == 'DT':
      tokenized_sentence.remove(word[0])
  for word in tokenized_sentence:
    if word == 'trump' or word =='biden' or word == 'joe' or word == 'donald' or word == ':':
      continue

    polarity_dict[word] = abs(Analyzer.polarity_scores(word)['compound'])

  polarity_dict = dict(sorted(polarity_dict.items(), key=lambda x: x[1], reverse = True))
  highest_valence_word = list(polarity_dict.keys())[0]
  masked_sentence = sentence.replace(highest_valence_word, "[MASK]")
  return sentence, masked_sentence

 

for sentence in validation_sentences['sentence']:
  print(sentence)
  sentence, masked_sentence = mask_word_in_sentence(sentence)
  print(sentence, masked_sentence)
  # masked_sentence = "Trump was [MASK] after the win of Obama."
  inputs = tokenizer(masked_sentence, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()
  avg_sentiment = 0
  for token in top_5_tokens:
    predicted_sentence = masked_sentence.replace(tokenizer.mask_token, tokenizer.decode([token]))
    print(f">>> {predicted_sentence}")
    if 'republicans' in predicted_sentence:
      predicted_sentence = predicted_sentence.replace('republicans',"")
      # print(predicted_sentence)
    sentiment = tsc.infer_from_text("", "republicans", predicted_sentence)
    # print(sentiment[0])
    
    if 'democrats' in predicted_sentence:
      predicted_sentence = predicted_sentence.replace('democrats',"")
      # print(predicted_sentence)
    sentiment = tsc.infer_from_text("", "democrats", predicted_sentence)
    # print(sentiment[0])
    if sentiment[0]['class_label'] == 'positive':
      avg_sentiment += sentiment[0]['class_prob']
    elif sentiment[0]['class_label'] == 'negative':
      avg_sentiment -= sentiment[0]['class_prob']
  avg_sentiment = avg_sentiment/5
  print(avg_sentiment)
  print()

  # if 'republicans' in sentence:
  #   sentence = sentence.replace('republicans',"")
  #   print(sentence)
  #   sentiment = tsc.infer_from_text("", "republicans", sentence)
  # if 'democrats' in sentence:
  #   sentence = sentence.replace('democrats',"")
  #   print(sentence)
  #   sentiment = tsc.infer_from_text("", "democrats", sentence)
  if avg_sentiment == 0:
    sentiments.append('neutral')
  else:
    sentiments.append(sentiment[0]['class_label'])
  sentiment_score.append(avg_sentiment)
  # break


In [64]:
validation_df

,sentence,masked_sentences,topics,target_sentences
0,republicans fires off tweets as us tackles cor...,republicans fires off tweets as us tackles cor...,virus_spreading,[MASK] fires off tweets as us tackles coronavi...
1,"on a call with republicans officials, police c...","on a call with republicans officials, police c...",preventive_measures,"on a call with [MASK] officials, police chief ..."
2,"coronavirus government response updates: us, r...","[MASK] government response updates: us, republ...",economy_education_impact,"coronavirus government response updates: us, [..."
3,coronavirus: republicans wants us open for bus...,coronavirus: republicans [MASK] us open for bu...,economy_education_impact,coronavirus: [MASK] wants us open for business...
4,republicans glad he is president during coron...,republicans glad he is president during coron...,politics,[MASK] glad he is president during coronaviru...
...,...,...,...,...
664,republicans snubs romney by inviting every oth...,republicans [MASK] romney by inviting every ot...,economy_education_impact,[MASK] snubs romney by inviting every other go...
665,"as testing outcry mounts, republicans cedes to...","as testing [MASK] mounts, republicans cedes to...",politics,"as testing outcry mounts, [MASK] cedes to stat..."
666,republicanss guidelines for reopening states a...,republicanss guidelines for reopening states a...,politics,[MASK]s guidelines for reopening states amid c...
667,reopening the economy depends on testing. repu...,[MASK] the economy depends on testing. republi...,politics,reopening the economy depends on testing. [MAS...


## Validating sentences with Target-Prior

Left leaning model predictions on Validation Dataset

In [16]:
%cd /content/drive/My Drive/Independent Study/validation/final/Left

/content/drive/My Drive/Independent Study/validation/final/Left


In [17]:
import numpy as np
L_target_democrats = []
L_target_republicans = []
L_target_trump = []

## TESTING
validation_sentences_df = pd.DataFrame()
validation_sentences_df = validation_groups[2]
count = 0
for prediction in validation_sentences_df['target_sentences']:
  print(count, "/",  len(validation_sentences_df['target_sentences']))
  count += 1
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  all_tokens = np.argsort(-mask_token_logits).tolist()

  word_probability_dict = {}
  for index, val in enumerate(mask_token_logits):
    word = tokenizer.decode([index])
    word_probability_dict[word] = val.numpy()
  print('Prediction:', prediction)
  print("Democrats prob. score:", word_probability_dict['democrats'])
  print("Republicans prob. score:", word_probability_dict['republicans'])
  # print("Trump prob. score:", word_probability_dict['trump'])
  print()
  # L_target_democrats.append(abs(word_probability_dict['democrats']))
  # L_target_republicans.append(abs(word_probability_dict['republicans']))
  # L_target_trump.append(abs(word_probability_dict['trump']))
  L_target_democrats.append(word_probability_dict['democrats'])
  L_target_republicans.append(word_probability_dict['republicans'])
  # L_target_trump.append(word_probability_dict['trump'])


L_prior_prob_dem = []
L_prior_prob_rep = []
count = 0
for i, prediction in enumerate(validation_sentences_df['masked_sentences']):
  print(count, "/",  len(validation_sentences_df['masked_sentences']))
  count += 1
  dem_prediction = ""
  rep_prediction = ""
  print(prediction)
  if "republicans" in prediction.lower():
    dem_prediction = prediction.lower().replace("republicans", "democrats").replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace('[mask]', '[MASK]')
  else:
    dem_prediction = prediction.lower().replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace("democrats", "republicans").replace('[mask]', '[MASK]')
  print(rep_prediction)
  print(dem_prediction)
  dem_inputs = tokenizer(dem_prediction, return_tensors="np")
  rep_inputs = tokenizer(rep_prediction, return_tensors="np")
  dem_token_logits = right_leaning_model(**dem_inputs).logits
  rep_token_logits = right_leaning_model(**rep_inputs).logits
  # Find the location of [MASK] and extract its logits
  dem_mask_token_index = np.argwhere(dem_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  dem_mask_token_logits = dem_token_logits[0, dem_mask_token_index, :]

  rep_mask_token_index = np.argwhere(rep_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  rep_mask_token_logits = rep_token_logits[0, rep_mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  dem_all_tokens = np.argsort(-dem_mask_token_logits).tolist()
  rep_all_tokens = np.argsort(-rep_mask_token_logits).tolist()

  dem_word_probability_dict = {}
  for index, val in enumerate(dem_mask_token_logits):
    word = tokenizer.decode([index])
    dem_word_probability_dict[word] = val.numpy()
  rep_word_probability_dict = {}
  for index, val in enumerate(rep_mask_token_logits):
    word = tokenizer.decode([index])
    rep_word_probability_dict[word] = val.numpy()
  L_prior_prob_dem.append(dem_word_probability_dict['democrats'])
  L_prior_prob_rep.append(rep_word_probability_dict['republicans'])
  print(dem_word_probability_dict['democrats'], rep_word_probability_dict['republicans'])
  # average_prob = (dem_word_probability_dict['democrats'] + rep_word_probability_dict['republicans']) / 2
  # B_prior_prob.append(abs(average_prob))
  print()
  # break

0 / 218
Prediction:  [MASK] glad he is president during coronavirus crisis
Democrats prob. score: 1.7592064
Republicans prob. score: 1.2492051

1 / 218
Prediction: democrats adviser slams president [MASK]s handling of coronavirus
Democrats prob. score: -2.8616786
Republicans prob. score: -2.902646

2 / 218
Prediction: sen. lindsey graham said hes listening to fauci and other coronavirus experts, not political punditry in a veiled jab at president [MASK]
Democrats prob. score: 2.4962084
Republicans prob. score: 2.7596323

3 / 218
Prediction: [MASK]s top infectious-disease expert described his battle to stop the president from making false coronavirus claims: i cant jump in front of the microphone and push him down
Democrats prob. score: -1.3311632
Republicans prob. score: -1.0252824

4 / 218
Prediction: [MASK] says spread of coronavirus not the fault of the asian american community
Democrats prob. score: -0.54995966
Republicans prob. score: -2.2968545

5 / 218
Prediction: democratic gro

Right leaning model predictions on Validation dataset

In [28]:
%cd /content/drive/My Drive/Independent Study/validation/final/Right

/content/drive/My Drive/Independent Study/validation/final/Right


In [43]:
import numpy as np
R_target_democrats = []
R_target_republicans = []
# R_target_trump = []

## TESTING
validation_sentences_df = pd.DataFrame()
validation_sentences_df = validation_groups[2]
count = 0
for prediction in validation_sentences_df['target_sentences']:
  print(count, "/",  len(validation_sentences_df['target_sentences']))
  count += 1
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = right_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  all_tokens = np.argsort(-mask_token_logits).tolist()

  word_probability_dict = {}
  for index, val in enumerate(mask_token_logits):
    word = tokenizer.decode([index])
    word_probability_dict[word] = val.numpy()
  print('Prediction:', prediction)
  print("Democrats prob. score:", word_probability_dict['democrats'])
  print("Republicans prob. score:", word_probability_dict['republicans'])
  # print("Trump prob. score:", word_probability_dict['trump'])
  print()
  # R_target_democrats.append(abs(word_probability_dict['democrats']))
  # R_target_republicans.append(abs(word_probability_dict['republicans']))
  # R_target_trump.append(abs(word_probability_dict['trump']))
  R_target_democrats.append(word_probability_dict['democrats'])
  R_target_republicans.append(word_probability_dict['republicans'])
  # R_target_trump.append(word_probability_dict['trump'])


R_prior_prob_dem = []
R_prior_prob_rep = []
count = 0
for i, prediction in enumerate(validation_sentences_df['masked_sentences']):
  print(count, "/",  len(validation_sentences_df['masked_sentences']))
  count += 1
  dem_prediction = ""
  rep_prediction = ""
  print(prediction)
  if "republicans" in prediction.lower():
    dem_prediction = prediction.lower().replace("republicans", "democrats").replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace('[mask]', '[MASK]')
  else:
    dem_prediction = prediction.lower().replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace("democrats", "republicans").replace('[mask]', '[MASK]')
  print(rep_prediction)
  print(dem_prediction)
  dem_inputs = tokenizer(dem_prediction, return_tensors="np")
  rep_inputs = tokenizer(rep_prediction, return_tensors="np")
  dem_token_logits = right_leaning_model(**dem_inputs).logits
  rep_token_logits = right_leaning_model(**rep_inputs).logits
  # Find the location of [MASK] and extract its logits
  dem_mask_token_index = np.argwhere(dem_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  dem_mask_token_logits = dem_token_logits[0, dem_mask_token_index, :]

  rep_mask_token_index = np.argwhere(rep_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  rep_mask_token_logits = rep_token_logits[0, rep_mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  dem_all_tokens = np.argsort(-dem_mask_token_logits).tolist()
  rep_all_tokens = np.argsort(-rep_mask_token_logits).tolist()

  dem_word_probability_dict = {}
  for index, val in enumerate(dem_mask_token_logits):
    word = tokenizer.decode([index])
    dem_word_probability_dict[word] = val.numpy()
  rep_word_probability_dict = {}
  for index, val in enumerate(rep_mask_token_logits):
    word = tokenizer.decode([index])
    rep_word_probability_dict[word] = val.numpy()
  R_prior_prob_dem.append(dem_word_probability_dict['democrats'])
  R_prior_prob_rep.append(rep_word_probability_dict['republicans'])
  print(dem_word_probability_dict['democrats'], rep_word_probability_dict['republicans'])
  # average_prob = (dem_word_probability_dict['democrats'] + rep_word_probability_dict['republicans']) / 2
  # B_prior_prob.append(abs(average_prob))
  print()
  # break

0 / 218
Prediction:  [MASK] glad he is president during coronavirus crisis
Democrats prob. score: 1.678721
Republicans prob. score: 1.1625717

1 / 218
Prediction: democrats adviser slams president [MASK]s handling of coronavirus
Democrats prob. score: -2.205729
Republicans prob. score: -2.384892

2 / 218
Prediction: sen. lindsey graham said hes listening to fauci and other coronavirus experts, not political punditry in a veiled jab at president [MASK]
Democrats prob. score: 3.6634412
Republicans prob. score: 2.7940316

3 / 218
Prediction: [MASK]s top infectious-disease expert described his battle to stop the president from making false coronavirus claims: i cant jump in front of the microphone and push him down
Democrats prob. score: -0.5589942
Republicans prob. score: -0.6205865

4 / 218
Prediction: [MASK] says spread of coronavirus not the fault of the asian american community
Democrats prob. score: 0.0537861
Republicans prob. score: -2.1017456

5 / 218
Prediction: democratic group l

Base model predictions

In [ ]:
import numpy as np
B_target_democrats = []
B_target_republicans = []
B_target_trump = []

## TESTING
validation_sentences_df = pd.DataFrame()
validation_sentences_df = validation_groups[2]
count = 0
for prediction in validation_sentences_df['target_sentences']:
  print(count, "/",  len(validation_sentences_df['target_sentences']))
  count += 1
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = base_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  all_tokens = np.argsort(-mask_token_logits).tolist()

  word_probability_dict = {}
  for index, val in enumerate(mask_token_logits):
    word = tokenizer.decode([index])
    word_probability_dict[word] = val.numpy()
  print('Prediction:', prediction)
  print("Democrats prob. score:", word_probability_dict['democrats'])
  print("Republicans prob. score:", word_probability_dict['republicans'])
  print("Trump prob. score:", word_probability_dict['trump'])
  print()
  # B_target_democrats.append(abs(word_probability_dict['democrats']))
  # B_target_republicans.append(abs(word_probability_dict['republicans']))
  # B_target_trump.append(abs(word_probability_dict['trump']))
  B_target_democrats.append(word_probability_dict['democrats'])
  B_target_republicans.append(word_probability_dict['republicans'])
  B_target_trump.append(word_probability_dict['trump'])


B_prior_prob_dem = []
B_prior_prob_rep = []
count = 0
for i, prediction in enumerate(validation_sentences_df['masked_sentences']):
  print(count, "/",  len(validation_sentences_df['masked_sentences']))
  count += 1
  dem_prediction = ""
  rep_prediction = ""
  print(prediction)
  if "republicans" in prediction.lower():
    dem_prediction = prediction.lower().replace("republicans", "democrats").replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace('[mask]', '[MASK]')
  else:
    dem_prediction = prediction.lower().replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace("democrats", "republicans").replace('[mask]', '[MASK]')
  print(rep_prediction)
  print(dem_prediction)
  dem_inputs = tokenizer(dem_prediction, return_tensors="np")
  rep_inputs = tokenizer(rep_prediction, return_tensors="np")
  dem_token_logits = right_leaning_model(**dem_inputs).logits
  rep_token_logits = right_leaning_model(**rep_inputs).logits
  # Find the location of [MASK] and extract its logits
  dem_mask_token_index = np.argwhere(dem_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  dem_mask_token_logits = dem_token_logits[0, dem_mask_token_index, :]

  rep_mask_token_index = np.argwhere(rep_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  rep_mask_token_logits = rep_token_logits[0, rep_mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  dem_all_tokens = np.argsort(-dem_mask_token_logits).tolist()
  rep_all_tokens = np.argsort(-rep_mask_token_logits).tolist()

  dem_word_probability_dict = {}
  for index, val in enumerate(dem_mask_token_logits):
    word = tokenizer.decode([index])
    dem_word_probability_dict[word] = val.numpy()
  rep_word_probability_dict = {}
  for index, val in enumerate(rep_mask_token_logits):
    word = tokenizer.decode([index])
    rep_word_probability_dict[word] = val.numpy()
  B_prior_prob_dem.append(dem_word_probability_dict['democrats'])
  B_prior_prob_rep.append(rep_word_probability_dict['republicans'])
  print(dem_word_probability_dict['democrats'], rep_word_probability_dict['republicans'])
  # average_prob = (dem_word_probability_dict['democrats'] + rep_word_probability_dict['republicans']) / 2
  # B_prior_prob.append(abs(average_prob))
  print()
  # break

In [44]:
validation_result_df = pd.DataFrame()
validation_result_df['org_sentence'] = validation_sentences_df['sentence']
# validation_result_df['L_target_democrats'] = L_target_democrats
# validation_result_df['L_target_republicans'] = L_target_republicans
# validation_result_df['L_prior'] = L_prior_prob

# validation_result_df['L_target_democrats'] = L_target_democrats
# validation_result_df['L_target_republicans'] = L_target_republicans
# validation_result_df['L_prior_dem'] = L_prior_prob_dem
# validation_result_df['L_prior_rep'] = L_prior_prob_rep

validation_result_df['R_target_democrats'] = R_target_democrats
validation_result_df['R_target_republicans'] = R_target_republicans
validation_result_df['R_prior_dem'] = R_prior_prob_dem
validation_result_df['R_prior_rep'] = R_prior_prob_rep

# validation_result_df['B_target_democrats'] = B_target_democrats
# validation_result_df['B_target_republicans'] = B_target_republicans
# validation_result_df['B_prior_dem'] = B_prior_prob_dem
# validation_result_df['B_prior_rep'] = B_prior_prob_rep

In [45]:
print(validation_result_df.sum())

org_sentence             republicans glad he is president during coron...
R_target_democrats                                             346.337891
R_target_republicans                                           189.585602
R_prior_dem                                                   -119.551498
R_prior_rep                                                   -236.426178
dtype: object


In [ ]:
dem_total_l = validation_result_df['L_target_democrats'].sum()
rep_total_l = validation_result_df['L_target_republicans'].sum()
prior_total_l = validation_result_df['L_prior'].sum()

dem_total_r = validation_result_df['R_target_democrats'].sum()
rep_total_r = validation_result_df['R_target_republicans'].sum()
prior_total_r = validation_result_df['R_prior'].sum()

import math
from decimal import *
  
# Initializing a decimal value
a = Decimal(abs(dem_total_l)/prior_total_l)
print ("Decimal value a : ", a)

b = Decimal(abs(rep_total_l)/prior_total_l)
  
print('The Association for Dem: ', a.log10())
print('The association for Rep: ', b.log10())

In [46]:
validation_result_df.to_csv (r'covid_validation_results_R_2.csv', index = False, header=True)